In [1]:
!pip install pyspark
!pip install pyarrow
!pip install -q findspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName("Linearreg").getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data=spark.read.csv("/content/Ecommerce_Customers.csv",inferSchema=True,header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'],
                            outputCol="features")

In [11]:
output=assembler.transform(data)

In [12]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [13]:
output.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|            features|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+--------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|[34.4972677251122...|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|[31.9262720263601...|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37

In [14]:
output.select("features").head(1)

[Row(features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [15]:
final_data=output.select(["features","Yearly Amount Spent"])

In [16]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [17]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [18]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                151|
|   mean| 504.06982096981136|
| stddev|   78.8913997630759|
|    min| 298.76200786180766|
|    max|  744.2218671047146|
+-------+-------------------+



In [19]:
lr=LinearRegression(labelCol="Yearly Amount Spent")

In [20]:
lr_model=lr.fit(train_data)

In [21]:
test_res=lr_model.evaluate(test_data)

In [22]:
test_res.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  12.884158953391761|
| -11.525959659419811|
| -2.7891479660227105|
|  23.666035065789174|
|   4.113893466761169|
|  20.487159515269752|
|   5.499046057393912|
|   4.558881179746493|
| -4.5091625834764955|
|   5.571024882780478|
| -0.4973564433763613|
|   18.21048874370473|
|-0.21198933977086654|
|  -12.94634604241719|
|  -8.024589618560867|
|   5.320335761115189|
|    24.0043140309088|
|   -5.68046079391479|
|  0.6590519249724593|
|   3.783756031606231|
+--------------------+
only showing top 20 rows



In [23]:
test_res.rootMeanSquaredError 

10.76376853670075

In [24]:
test_res.meanSquaredError

115.858713111669

In [25]:
test_res.r2

0.9812606266012187

In [26]:
unlabeled_data=test_data.select("features")

In [27]:
res=lr_model.transform(unlabeled_data)

In [28]:
res.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 395.7561921192357|
|[31.0662181616375...|460.45925286709416|
|[31.2681042107507...|426.25968113984663|
|[31.2834474760581...| 568.1150543598783|
|[31.3091926408918...|428.60682437317246|
|[31.3123495994443...|443.10425851267087|
|[31.3584771924370...| 489.6769043920815|
|[31.4459724827577...| 480.3180837553821|
|[31.5257524169682...| 448.4747893933584|
|[31.5316044825729...| 430.9445808465821|
|[31.5761319713222...| 541.7239404327047|
|[31.6098395733896...|426.33506090740343|
|[31.8627411090001...| 556.5101305138176|
|[31.9365486184489...| 440.1457309377454|
|[32.0085045178551...|451.22181064731626|
|[32.0215955013870...| 516.2518389967122|
|[32.0498393904573...| 454.7150428433065|
|[32.0883806304482...| 517.8463271820729|
|[32.0961089938451...| 374.7394034852707|
|[32.1164001209281...|459.70123937265976|
+--------------------+------------